In [1]:
import pandas as pd
import json
import sys
import numpy as np
sys.path.append('..')
from helpers.funcs import *

In [2]:
config = json.load(open('../config.json', 'r'))
INPUT_DIR = config['INPUT_DIR']
OUTPUT_DIR = config['OUTPUT_DIR']
TWEET_DIR = config['TWEET_DIR']
NUM_CLUSTERS = config['NUM_CLUSTERS']
events = open(INPUT_DIR + 'event_names.txt', 'r').read().splitlines()

In [3]:
data_filter_method = '_noRT' # make sure to include underscore
#data_filter_method = '_clustered'
#cluster_method = '_relative'
cluster_method = ''

In [4]:
time_dict = json.load(open(INPUT_DIR + "event_year.json","r"))
shooter_race = json.load(open(INPUT_DIR + "shooters_race.json","r"))
polarization_dict = json.load(open(OUTPUT_DIR + "polarization"+data_filter_method+ cluster_method+".json","r"))

In [ ]:
between_topic_pol = json.load(open(DATA_DIR + "between_topic_polarization"+cluster_method+".json","r"))

## for plotting overall polarization

In [ ]:
x = []
y = []
y_random = []
labels=[]
ex = ['fort_lauderdale']  # we exclude Fort Lauderdale because we only have data for the first day after the shooting
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    x_val = float(2000+t)
    y_val = float(polarization_dict[e][0])
    y_random_val = float(float(polarization_dict[e][1]))
    x.append(x_val)
    y.append(y_val)
    y_random.append(y_random_val)
    labels.append(new_label)
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y + y_random), 'label':labels * 2, 'is_actual':['actual value']* len(y) + ['value resulting from random party assignment']* len(y)})
df.to_csv(DATA_DIR + "polarization"+data_filter_method+cluster_method+".csv", index=False)

## for plotting within vs between topic polarization

In [ ]:
x = []
y_between = []
y_within = []
labels=[]
race = []
ex = ['fort_lauderdale']
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    within_topic_pol = json.load(open(TWEET_DIR +e+"/"+e+"_topic_polarization"+cluster_method+".json","r"))
    
    # get the topic proportions
    topics = pd.read_csv(TWEET_DIR + event + '/' + event + '_kmeans_topics_' + str(NUM_CLUSTERS) + '.csv')
    if cluster_method != '':
        assigned_indices, topics = get_assigned_indices_relative(topics) if cluster_method == '_relative' else get_assigned_indices_absolute(topics)
        
    print(e, len(topics))
    within_topic_pol_val = (np.array([float(within_topic_pol[str(i)][0]) for i in range(NUM_CLUSTERS)]) * np.bincount(topics['topic_0'])).sum() / len(topics)
    
    x_val = float(2000+t)
    y_between_val = float(between_topic_pol[e][0])
    y_within_val = within_topic_pol_val
    x.append(x_val)
    y_between.append(y_between_val)
    y_within.append(y_within_val)
    labels.append(new_label)
    race.append(shooter_race[e])
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y_between + y_within), 'label':labels * 2, 'kind':['between-topic']* len(y_between) + ['within-topic']* len(y_within), 'race':race * 2})
df.to_csv(DATA_DIR + "topic_polarization"+cluster_method+".csv", index=False)

## combine polarization over time values

In [6]:
dicts = []
for e in events:
    vals = np.load(TWEET_DIR + e + '/' + e + '_polarization_over_time_log_noRT.npy')
    for i in range(vals.shape[0]):
        d = {}
        d['event'] = e
        d['leaveout'] = vals[i, 0]
        d['time'] = vals[i, 3]
        d['size'] = vals[i, 2]
        d['squared_diff'] = np.abs(vals[i, 1] - .5)
        dicts.append(d)
df = pd.DataFrame(dicts)
df.to_csv(OUTPUT_DIR + 'polarization_over_time_log.csv', index=False)

## combine user leaveout scores (this is not used in the paper)

In [6]:
dfs = []
for e in events:
    df = pd.read_csv(TWEET_DIR + e + '/' + e + '_user_leaveout.csv')
    df['event'] = e
    dfs.append(df)

In [7]:
new = pd.concat(dfs, ignore_index=True)

In [9]:
new.to_csv(TWEET_DIR + 'user_leaveouts.csv', index=False)